# Исследование данных с результатами опроса, отражающего текущий уровень потребительской лояльности (NPS) клиентов телекоммуникационной компании.

Заказчик - телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.
Перед компанией стоит задача определить текущий уровень потребительской лояльности - NPS среди клиентов из России.
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и потребовалось подготовить дашборд с его итогами.
Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
path_to_db = 'telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT  u.user_id,
        u.lt_day,
        CASE
            WHEN u.lt_day < 366 THEN 'True'
            ELSE 'False'
        END AS is_new,
        u.age,
        CASE
            WHEN u.gender_segment == 1 THEN 'Female'
            WHEN u.gender_segment == 0 THEN 'Male'
        END AS gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        TRIM(SUBSTR(a_s.title, 3)) AS age_segment,
        TRIM(SUBSTR(t_s.title, 3)) AS traffic_segment,
        TRIM(SUBSTR(l_s.title, 3)) AS lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score < 7 THEN 'критики'
            WHEN u.nps_score > 6 AND u.nps_score < 9 THEN 'нейтралы'
            WHEN u.nps_score > 8 THEN 'сторонники'
        END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS a_s ON u.age_gr_id = a_s.age_gr_id
LEFT JOIN traffic_segment AS t_s ON u.tr_gr_id = t_s.tr_gr_id
LEFT JOIN lifetime_segment AS l_s ON u.lt_gr_id = l_s.lt_gr_id;
"""
try:
    df = pd.read_sql(query, engine)
    print('Данные загружены')
except:
    print('Не удалось загрузить данные')

Данные загружены


In [4]:
df.sample(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
290291,P0IDVC,567,False,44.0,Female,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,75-80,13-24,10,сторонники
353399,SAA12X,97,True,42.0,Female,ANDROID,SMARTPHONE,Россия,Барнаул,35-44,35-40,4-6,10,сторонники
243269,MKW4LN,4738,False,33.0,Male,ANDROID,SMARTPHONE,Россия,Екатеринбург,25-34,80-85,36+,10,сторонники
436013,WK497P,521,False,26.0,Male,ANDROID,SMARTPHONE,Россия,Волгоград,25-34,10-15,13-24,9,сторонники
459526,XRWUBD,1519,False,32.0,Female,ANDROID,SMARTPHONE,Россия,Тольятти,25-34,100+,36+,10,сторонники
288783,OXRHUT,1047,False,23.0,Female,ANDROID,SMARTPHONE,Россия,Краснодар,16-24,5-10,25-36,8,нейтралы
279966,OHEJTC,4406,False,45.0,Female,ANDROID,SMARTPHONE,Россия,Хабаровск,45-54,10-15,36+,3,критики
37738,BY9FVM,477,False,50.0,Male,ANDROID,SMARTPHONE,Россия,Новокузнецк,45-54,1-5,13-24,8,нейтралы
187815,JPIAS9,4413,False,52.0,Female,ANDROID,SMARTPHONE,Россия,НижнийТагил,45-54,20-25,36+,10,сторонники
36689,BWAIHK,1743,False,32.0,Female,ANDROID,SMARTPHONE,Россия,Сочи,25-34,15-20,36+,5,критики


In [5]:
try:
    df.to_csv('telecomm_csi_tableau.csv', index=False)
    print('Данные выгружены в файл telecomm_csi_tableau.csv')
except:
    print('Не удалось выгрузить данные')

Данные выгружены в файл telecomm_csi_tableau.csv


**Ссылка на дашборд tableau**

https://public.tableau.com/app/profile/yury.gnezdilov/viz/telecomm_csi_tableau_16474468079970/3_PROMOTERS

**Ссылка на все файлы проекта**

https://github.com/ygnezdilov/telecomm_csi/